In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
import string
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from imblearn.over_sampling import SMOTE
from numpy import where
from numpy.lib.function_base import append
from operator import itemgetter
from sklearn.metrics import confusion_matrix
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from sklearn.naive_bayes import MultinomialNB

In [4]:
mylink = 'https://docs.google.com/spreadsheets/d/1dSrfhPfyvrnZMvsX2-qxDTLRt7CjVn7m/edit?usp=sharing&ouid=103635145999297036605&rtpof=true&sd=true'
path = 'https://drive.google.com/uc?export=download&id='+mylink.split('/')[-2]
df = pd.read_excel(path)
df.head()

,NO,NAMA,LIRIK,ASAL,SUMBER,ASAL2,ASAL3
0,1.0,Bungong Jeumpa,Bungong Jeumpa Bungong Jeumpa Meugah di Aceh ...,Aceh,Buku Kumpulan Lagu-lagu Daerah Nusantara,Daerah,Barat
1,2.0,Jambo – Jambo,"Jambo-jambo, Hai jambo sinan diblang Jambo-j...",Aceh,https://www.bercamilan.my.id/2020/09/lirik-lag...,Daerah,Barat
2,3.0,Lon Sayang,Daerah Aceh tanoh lon sayang nibak tempat nya...,Aceh,https://lagudaerah.id/lon-sayang/,Daerah,Barat
3,4.0,Tawar Sadenge,Engon ko so tanoh Gayo Si megah mu reta dele ...,Aceh,https://lagudaerah.id/tawar-sadenge/,Daerah,Barat
4,5.0,Aneuk Yatim,Jinoe lon kisah saboh riwayat Kisah baro that...,Aceh,https://lagudaerah.id/aneuk-yatim/,Daerah,Barat


In [6]:
asal=['ASAL','ASAL2','ASAL3']
score = pd.DataFrame({"asal":[],"ksmote":[],"akurasi":[],"presisi":[],"recall":[]})
for asl in asal:
  ww=df[asl]
  kk=ww.unique()
  kk
  x=0
  jj=[]
  for b in kk:
    jj.append(x)
    x=x+1
  newkk=ww.replace(kk,jj)
    # define dataset
  data = df['LIRIK']
  pipeline = Pipeline([
      ('vect', CountVectorizer()),
      ('tfidf', TfidfTransformer(use_idf=False)),
  ])        
  X = pipeline.fit_transform(data).todense()
  y=newkk
  # summarize class distribution

  fd=10
  #ks=[89]
  #knn=[2,3,4,5,6,7,8,9,10]
  kf= KFold(n_splits=fd, random_state=0, shuffle=True)

  X1=X
  y1=y
  # transform the dataset

  # summarize the new class distribution
  counter = Counter(y)
  min_key, min_count = min(counter.items(), key=itemgetter(1))
  print(min_count)
  for j in range(90):
    if j<=0:
      j='-'
    else:
      if j>=min_count:
        break
      else:
        oversample = SMOTE(k_neighbors=j)
        X, y = oversample.fit_resample(X1, y1)
        #counter = Counter(y)
        #print(counter)
    neigh2 = MultinomialNB()
    avg_acc = np.zeros(fd)
    avg_pre = np.zeros(fd)
    avg_rec = np.zeros(fd)
    i = 0
    for train_index, test_index in kf.split(X):    
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]
      neigh2.fit(X_train,y_train)
      y_pred = neigh2.predict(X_test)
      acc = accuracy_score(y_test, y_pred)
      pre = precision_score(y_test, y_pred, average='macro')
      rec = recall_score(y_test, y_pred, average='macro')
      avg_acc[i] = acc
      avg_pre[i] = pre
      avg_rec[i] = rec
      i = i+1
    score = score.append({"asal":str(asl),"ksmote":str(j),"akurasi":np.average(avg_acc)*100,"presisi":np.average(avg_pre)*100,"recall":np.average(avg_rec)*100},ignore_index=True)
      #print(confusion_matrix(y_test, y_pred)) 
  # for appending df2 at the end of df1
  #print("k="+str(k)+",smote="+str(j))
  #print("Akurasi : ", np.average(avg_acc))
  #print("Presisi : ", np.average(avg_pre))
  #print("Recall : ", np.average(avg_rec))
  #scatter plot of examples by class label
  #pca = PCA(n_components=2).fit(X)
  #data2D = pca.transform(X)
  #plt.scatter(data2D[:,0], data2D[:,1],c=y)
  #plt.show()
  print(counter)
score


4
Counter({30: 90, 8: 42, 27: 41, 26: 34, 28: 28, 17: 26, 25: 20, 10: 19, 4: 17, 20: 17, 7: 16, 9: 15, 21: 15, 1: 14, 23: 14, 6: 13, 16: 13, 13: 12, 5: 11, 22: 11, 29: 11, 0: 10, 2: 10, 3: 10, 11: 10, 12: 10, 14: 10, 18: 10, 19: 10, 24: 7, 15: 4})
90
Counter({0: 480, 1: 90})
43
Counter({0: 316, 1: 121, 3: 90, 2: 43})


,asal,ksmote,akurasi,presisi,recall
0,ASAL,-,25.964912,13.983789,11.546961
1,ASAL,1,98.136201,97.982920,98.291440
2,ASAL,2,98.494624,98.385615,98.582413
3,ASAL,3,98.422939,98.331803,98.554930
4,ASAL2,-,90.350877,94.856034,69.335165
...,...,...,...,...,...
132,ASAL3,38,91.373578,91.898804,91.512753
133,ASAL3,39,91.216098,91.794860,91.271172
134,ASAL3,40,91.611674,92.153579,91.708228
135,ASAL3,41,91.849769,92.385583,91.909166


In [7]:
score.to_excel(r'hasilmnb.xlsx', index=False)